URL:

https://jahazielponce.com/trucos-pandas-dataframe

## Leyendo un CSV

In [1]:
import gc

In [2]:
csv_FILENAME = 'D:/10_PUBLICACIONES/publicaciones/Trucos_pandas/train.csv'

In [3]:
import pandas as pd

In [4]:
%%time
pandas_df = pd.read_csv(csv_FILENAME)

Wall time: 28.1 s


In [5]:
import datatable as dt
import dask.dataframe as dd
import vaex

C:\Users\jahaz\.conda\envs\jps_ml_models\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\jahaz\.conda\envs\jps_ml_models\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\jahaz\.conda\envs\jps_ml_models\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
C:\Users\jahaz\.conda\envs\jps_ml_models\lib\importlib\_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)


In [6]:
%%time
datatable_df = dt.fread(csv_FILENAME).to_pandas()

Wall time: 1.98 s


In [7]:
%%time
dask_df = dd.read_csv(csv_FILENAME).compute()

Wall time: 12.2 s


In [8]:
%%time
vaex_df = vaex.from_csv(csv_FILENAME).to_pandas_df()

Wall time: 30.5 s


In [9]:
del datatable_df
gc.collect()

570

In [10]:
del dask_df
gc.collect()

20

In [11]:
del vaex_df
gc.collect()

20

## Método Apply

In [12]:
import numpy as np

In [13]:
def genera_variables(col1, col2, col3, col4):
    return np.log1p((col1 + col3) ** 2 / np.sqrt(np.abs(col2) + np.abs(col4)))

In [14]:
%%time
pandas_df['nueva_variable'] = pandas_df.apply(
    lambda row: genera_variables(
        row['f1'], 
        row['f2'], 
        row['f3'], 
        row['f4']
    ), 
    axis=1
)

Wall time: 52.6 s


In [15]:
%%time
pandas_df['nueva_variable_np'] = genera_variables(pandas_df['f1'].values,
                                                  pandas_df['f2'].values,
                                                  pandas_df['f3'].values,
                                                  pandas_df['f4'].values)

Wall time: 101 ms


In [16]:
%%time
pandas_df['nueva_variable_list'] = [genera_variables(f1, f2, f3, f4) for (f1, f2, f3, f4) in zip(pandas_df['f1'],
                                                                                                 pandas_df['f2'],
                                                                                                 pandas_df['f3'],
                                                                                                 pandas_df['f4'])]

Wall time: 12.9 s


In [17]:
pandas_df[pandas_df['nueva_variable'] != pandas_df['nueva_variable_np']][['nueva_variable', 'nueva_variable_np']].iloc[0, 0]

1.2274606978527784

In [18]:
pandas_df[pandas_df['nueva_variable'] != pandas_df['nueva_variable_np']][['nueva_variable', 'nueva_variable_np']].iloc[0, 1]

1.2274606978527787

In [19]:
(pandas_df['nueva_variable'] != pandas_df['nueva_variable_list']).sum()

0

## Optimización de memoria

In [20]:
def reduciendo_uso_memoria(df: pd.DataFrame, verbose: bool = True) -> pd.DataFrame:
    DICT_TIPO_INT = {
        "int8": [np.iinfo(np.int8).min, np.iinfo(np.int8).max],
        "int16": [np.iinfo(np.int16).min, np.iinfo(np.int16).max],
        "int32": [np.iinfo(np.int32).min, np.iinfo(np.int32).max],
        "int64": [np.iinfo(np.int64).min, np.iinfo(np.int64).max]
    }
    DICT_TIPO_FLOAT = {
        "float16": [np.finfo(np.float16).min, np.finfo(np.float16).max],
        "float32": [np.finfo(np.float32).min, np.finfo(np.float32).max]
    }
    
    LIST_TYPES = list(DICT_TIPO_INT.keys()) + list(DICT_TIPO_FLOAT.keys()) + ['float64']
    
    memoria_inicial = df.memory_usage().sum() / 1024 ** 2
    
    for col in df.columns:
        col_type = df[col].dtypes
        
        if col_type not in LIST_TYPES:
            continue
        
        c_min, c_max = df[col].min(), df[col].max()
        
        if str(col_type)[:3] == "int":
            if c_min > DICT_TIPO_INT["int8"][0] and c_max < DICT_TIPO_INT["int8"][1]:
                df[col] = df[col].astype(np.int8)
            elif c_min > DICT_TIPO_INT["int16"][0] and c_max < DICT_TIPO_INT["int16"][1]:
                df[col] = df[col].astype(np.int16)
            elif c_min > DICT_TIPO_INT["int32"][0] and c_max < DICT_TIPO_INT["int32"][1]:
                df[col] = df[col].astype(np.int32)
            elif c_min > DICT_TIPO_INT["int64"][0] and c_max < DICT_TIPO_INT["int64"][1]:
                df[col] = df[col].astype(np.int64)
        else:
            if c_min > DICT_TIPO_FLOAT["float16"][0] and c_max < DICT_TIPO_FLOAT["float16"][1]:
                df[col] = df[col].astype(np.float16)
            elif c_min > DICT_TIPO_FLOAT["float32"][0] and c_max < DICT_TIPO_FLOAT["float32"][1]:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
    memoria_final = df.memory_usage().sum() / 1024 ** 2
    
    if verbose:
        print("Uso de memoria reducido a {:.2f} Mb (reducción {:.1f}%)".format(
            memoria_final, 100 * (memoria_inicial - memoria_final) / memoria_inicial
        )
             )
    return df

In [21]:
reduciendo_uso_memoria(pandas_df)

Uso de memoria reducido a 615.12 Mb (reducción 74.6%)


,Unnamed: 0,id,f0,f1,f2,f3,f4,f5,f6,f7,...,f94,f95,f96,f97,f98,f99,target,nueva_variable,nueva_variable_np,nueva_variable_list
0,0,0,0.106628,3.593750,132.7500,3.183594,0.081970,1.188477,3.732422,2.265625,...,0.052765,0.065430,4.210938,1.978516,0.085999,0.240479,0,1.606445,1.606445,1.606445
1,1,1,0.125000,1.673828,76.5625,3.378906,0.099426,5.093750,1.275391,-0.471436,...,0.154053,0.606934,-0.267822,2.578125,-0.020874,0.024719,0,1.365234,1.365234,1.365234
2,2,2,0.036316,1.497070,233.5000,2.195312,0.026917,3.126953,5.058594,3.849609,...,0.116089,-0.001689,-0.520020,2.140625,0.124451,0.148193,0,0.637695,0.637695,0.637695
3,3,3,-0.014076,0.245972,780.0000,1.890625,0.006947,1.531250,2.697266,4.515625,...,0.062012,0.041199,0.511719,1.968750,0.040009,0.044861,0,0.151367,0.151367,0.151367
4,4,4,-0.003260,3.714844,156.1250,2.148438,0.018280,2.097656,4.156250,-0.038239,...,0.125122,0.037506,1.043945,1.075195,-0.012817,0.072815,1,1.322266,1.322266,1.322266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999995,599995,599995,0.042999,1.640625,1376.0000,1.086914,0.839355,1.693359,3.843750,1.108398,...,0.225098,0.073181,4.437500,1.739258,0.049042,0.065796,1,0.182739,0.182739,0.182739
2999996,599996,599996,0.137085,4.828125,-99.6250,0.985352,0.037720,3.302734,2.472656,5.941406,...,0.234009,0.041382,3.873047,1.870117,0.162354,0.085632,0,1.477539,1.477539,1.477539
2999997,599997,599997,1.316406,0.501953,66.2500,2.232422,0.060028,3.773438,2.656250,3.156250,...,0.027466,0.061920,-0.383301,0.921875,0.084839,0.052643,0,0.651855,0.651855,0.651855
2999998,599998,599998,0.511719,3.828125,778.0000,0.285889,0.520020,2.763672,2.521484,1.561523,...,-0.038086,0.038818,-0.122864,3.107422,0.061340,0.004177,1,0.474121,0.474121,0.474121


## Guardando en CSV

In [22]:
FILENAME_SAVE_CSV = 'D:/10_PUBLICACIONES/publicaciones/Trucos_pandas/final.csv'
FILENAME_SAVE_FEATHER = 'D:/10_PUBLICACIONES/publicaciones/Trucos_pandas/final.feather'
FILENAME_SAVE_PICKLE = 'D:/10_PUBLICACIONES/publicaciones/Trucos_pandas/final.pkl'

In [23]:
%%time
pandas_df.to_csv(FILENAME_SAVE_CSV, index=False)

Wall time: 3min 52s


In [24]:
%%time
pandas_df.to_feather(FILENAME_SAVE_FEATHER)

Wall time: 559 ms


In [25]:
%%time
pandas_df.to_pickle(FILENAME_SAVE_PICKLE)

Wall time: 7.12 s
